In [1]:
import pandas as pd

# Merging the same infobox
excel_path = pd.ExcelFile('D:\Kuliah\Skripsi\Ontologi-Sejarah-Indonesia\MichaelDawBalma\Code\infoboxes\infobox-orla.xlsx')
sheet_name = pd.read_excel(excel_path, 'military conflict1')
sheet_name2 = pd.read_excel(excel_path, 'Military Conflict2')
sheet_name3 = pd.read_excel(excel_path, 'military conflict')
csv_file_path = 'military-conflict-final.csv'

merged_df = pd.concat([sheet_name, sheet_name2, sheet_name3])

merged_df.to_csv(csv_file_path, index=False)

In [4]:
# csv.field_size_limit(100000000)

131072

In [3]:
import csv
military_conflict_clean = []

with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        military_conflict_clean.append(dict(row))

In [36]:
import geopy

def get_coordinates(place):
    geolocator = geopy.Nominatim(user_agent='indonesian-history-ontology')
    lct = geolocator.geocode(place, geometry='wkt')
    if lct:
        return lct.latitude, lct.longitude, lct.raw['geotext']
    else:
        return -0.789275, 113.921327, ''


for index, value in enumerate(military_conflict_clean):
    latitude, longitude, location = get_coordinates(value['place'])

    # Add latitude and longitude into the dictionary  
    value['latitude'] = latitude
    value['longitude'] = longitude
    value['location'] = location

In [65]:
'''
Merge column date_start-year_start and date_end-year_end into merged_date_start and merged_date_end
https://raw.githubusercontent.com/w3c/sdw/gh-pages/time/rdf/time.ttl (time:inXSDDate)
'''

month_list = {
    'Januari' : '01',
    'Februari' : '02',
    'Maret' : '03',
    'April' : '04',
    'Mei' : '05',
    'Juni' : '06',
    'Juli' : '07',
    'Agustus' : '08',
    'September' : '09',
    'Oktober' : '10',
    'November' : '11',
    'Desember' : '12',
}

defaultDateStart = '01-01'
defaultDateEnd = '12-31'

def check_date_format(date):
    # returns to dd format
    if len(date) == 1:
        return '0' + f'{date}'
    return date

for index, value in enumerate(military_conflict_clean):
    
    day = ''
    month = ''
    year = ''
    dateStartUsed = ''
    dateEndUsed = ''
    dt = ''
    dtSt = ''
    dtEnd = ''

    split_date = value['date'].split("-")
    
    # if it does not have end date
    if len(split_date) == 1 :
        dt = split_date[0].strip().split(" ")
        # if the dt is only the year
        if len(dt) == 1 :
            dateStartUsed = f"{dt[0]}-{defaultDateStart}"
            dateEndUsed = f"{dt[0]}-{defaultDateEnd}"
        else:
            day = dt[0]
            month = month_list[dt[1]]
            year = dt[2]
            
            day = check_date_format(day)
            dateStartUsed = f'{year}-{month}-{day}'
            dateEndUsed = dateStartUsed
            
    # if it does have end date
    else :
        dtSt = split_date[0].strip().split(" ")
        dtEnd = split_date[1].strip().split(" ")
        
        # end date were in good format
        day = dtEnd[0]
        month = month_list[dtEnd[1]]
        year = dtEnd[2]
        # change into dd format
        day = check_date_format(day)
        dateEndUsed = f"{year}-{month}-{day}"
        
        # start date and it is year only
        if len(dtSt) == 1:
            dateStartUsed = f"{dtSt[0]}-{defaultDateStart}"
        # start date without year; get end-date's year
        elif len(dtSt) == 2:
            day = check_date_format(dtSt[0])
            month = month_list[dtSt[1]]
            dateStartUsed = f"{dtEnd[2]}-{month}-{day}"
        else :
            day = check_date_format(dtSt[0])
            month = month_list[dtSt[1]]
            dateStartUsed = f"{dtSt[2]}-{month}-{day}"
    
    value['start-date'] = dateStartUsed
    value['end-date'] = dateEndUsed


In [72]:
import wikipedia

def retrieve_paragraph(article_title):
    wikipedia.set_lang("id")
    return wikipedia.summary(article_title, auto_suggest=False)

def get_url_wikipedia(article_title):
    wikipedia.set_lang("id")
    page = wikipedia.page(article_title, auto_suggest=False)
    return page.url


In [75]:
for index, data in enumerate(military_conflict_clean[:10]):
    # if the title has more than paragraph summary, get the first summary
    summary = retrieve_paragraph(data['page_title']).split("\n")
    data['summary'] = summary[0]
    
    wikiurl = get_url_wikipedia(data['page_title'])
    data['wikiurl'] = wikiurl

In [76]:
for index, data in enumerate(military_conflict_clean[10:]):
    # if the title has more than paragraph summary, get the first summary
    summary = retrieve_paragraph(data['page_title']).split("\n")
    data['summary'] = summary[0]

    wikiurl = get_url_wikipedia(data['page_title'])
    data['wikiurl'] = wikiurl

In [84]:
fieldnames = military_conflict_clean[0].keys()

with open('military-conflict-final-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    writer.writerows(military_conflict_clean)

### Handling Actor on Multivalue Commander 

In [52]:
# Handling page_title, wikiurl, and summary (if any) for commanders from a 'Military Conflict' Event. (For Person mapping in timeline)
# Birth-Death Date and image will be handled manually.

import csv
commander1 = []
commander2 = []

with open('commander1.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        commander1.append(dict(row))

with open('commander2.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        commander2.append(dict(row))



In [53]:
import wikipedia

for value in commander1:
    if value['commander1'] != '' :
        wikipedia.set_lang("id")
        try:
            page = wikipedia.page(value['commander1'], auto_suggest=False)
            value['page_title'] = page.title
            value['summary'] = page.summary.split("\n")[0]
            value['wikiurl'] = page.url
        except wikipedia.PageError :
            value['page_title'] = ''
            value['summary'] = ''
            value['wikiurl'] = ''

for value in commander2:
    if value['commander2'] != '' :
        wikipedia.set_lang("id")
        try:
            page = wikipedia.page(value['commander2'], auto_suggest=False)
            value['page_title'] = page.title
            value['summary'] = page.summary.split("\n")[0]
            value['wikiurl'] = page.url
        except wikipedia.PageError :
            value['page_title'] = ''
            value['summary'] = ''
            value['wikiurl'] = ''

In [54]:
fieldnames = commander1[0].keys()
with open('commander1-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(commander1)

In [55]:
fieldnames2 = commander2[0].keys()
with open('commander2-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames2)
    writer.writeheader()
    writer.writerows(commander2)

### Handle Vocab Changes and Merging Vocab

In [14]:
import csv
csv.field_size_limit(100000000)
military_conflict_clean = []

with open('military-conflict-final-cleaned.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        military_conflict_clean.append(dict(row))

In [15]:
for military_conflict in military_conflict_clean :
    casualties = []
    if military_conflict['casualties1']:
        casualties.append(military_conflict['casualties1'])
    if military_conflict['casualties2']:
        casualties.append(military_conflict['casualties2'])
    if military_conflict['casualties3']:
        casualties.append(military_conflict['casualties3'])
    military_conflict['casualties'] = ', '.join(casualties)
    
    del military_conflict['casualties1']
    del military_conflict['casualties2']
    del military_conflict['casualties3']


In [16]:
fieldnames = military_conflict_clean[0].keys()

with open('military-conflict-final-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    writer.writerows(military_conflict_clean)

In [17]:
# Combatant
combatant1 = []
combatant2 = []
with open('combatant1.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        combatant1.append(dict(row))
with open('combatant2.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        combatant2.append(dict(row))

In [30]:
merged_list = []

for f_entry, s_entry in zip(combatant1, combatant2):
    key = f_entry['key']
    side = ', '.join(filter(None, [f_entry['combatant1'], s_entry['combatant2']]))
    merged_list.append({'key': key, 'side': side})

In [32]:
fieldnames = ['key', 'side']

with open('combatant-final.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for entry in merged_list:
        writer.writerow(entry)

### Redo Place

In [5]:
import csv

file_path = 'military-conflict-final-cleaned.csv'
final_data = []

with open(file_path, mode='r', encoding='utf-8', newline='') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        final_data.append(row)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [28]:
title_place = []
place = []

for index, value in enumerate(final_data):
    title_place.append(value[8])
    place.append(value[2])
         

['page_title', 'Pemberontakan DI/TII di Aceh', 'Pertempuran Lima Hari', 'Pertempuran Bojong Kokosan', 'Pertempuran Lengkong', 'Bandung Lautan Api', 'Perang Cumbok', 'Delapan Jam Pertempuran di Mangkang', 'Puputan Margarana', 'Perang Dayak Desa', 'Gerakan 30 September', 'Kampanye Kalimantan (1945)', 'Pertempuran Medan Area', 'Serangan Umum Surakarta', 'Serangan Umum 1 Maret 1949', 'Penyerbuan Kotabaru Yogyakarta', 'Pertempuran Surabaya', 'Peristiwa 19 November', 'Peristiwa Tiga Maret', 'Pertempuran Kumai', 'Revolusi Nasional Indonesia', 'Kampanye Hindia Belanda', 'Operasi 17 Agustus', 'Konflik Papua']
['place', 'Aceh', 'Semarang, Jawa Tengah, Indonesia', 'Sukabumi', 'Desa Lengkong, Serpong, Tangerang Selatan', 'Bandung, Jawa Barat, Indonesia', 'Kabupaten Pidie, Aceh', 'Mangkang, Semarang Indonesia', 'Desa Marga, Kecamatan Marga, Tabanan, Bali, Indonesia', 'Sanggau, Kalimantan Barat', 'Kelurahan Lubang Buaya, Cipayung, Jakarta Timur', 'Kalimantan', 'Medan, Sumatera Utara, Indonesia', 'Su

In [30]:
# Combine title_place and place into a list of rows
rows = zip(title_place, place)

# Specify the column names
fieldnames = ['key', 'place']

# Specify the output file path
output_file = 'final-place-military-conflict.csv'

# Write the data to a new CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(fieldnames)
    # Write the rows
    writer.writerows(rows)

print(f"Data has been written to {output_file}")

Data has been written to final-place-military-conflict.csv


### Redo Date

In [31]:
import csv

file_path = 'military-conflict-final.csv'
final_data = []

with open(file_path, mode='r', encoding='utf-8', newline='') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        final_data.append(row)

In [64]:
title_date = []
date = []

for index, value in enumerate(final_data):
    title_date.append(value[8])
    date.append(value[3])

In [65]:
# Combine title_place and place into a list of rows
rows = zip(title_date, date)

# Specify the column names
fieldnames = ['key', 'start_date']

# Specify the output file path
output_file = 'final-date-military-conflict.csv'

# Write the data to a new CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(fieldnames)
    # Write the rows
    writer.writerows(rows)

print(f"Data has been written to {output_file}")

Data has been written to final-date-military-conflict.csv


### Redo Date Commander

In [66]:
import csv

file_path = 'commander-final.csv'
final_data = []

with open(file_path, mode='r', encoding='utf-8', newline='') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        final_data.append(row)

In [92]:
title_date = []
alt = []
start_date = []
end_date = []

for index, value in enumerate(final_data):
    title_date.append(value[2])
    alt.append(value[1])
    start_date.append(value[5])
    end_date.append(value[6])

In [93]:
# Combine title_place and place into a list of rows
rows = zip(title_date, alt, start_date, end_date)

# Specify the column names
fieldnames = ['mainkey', 'key', 'start_date', 'end_date']

# Specify the output file path
output_file = 'final-date-military-conflict-commander.csv'

# Write the data to a new CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(fieldnames)
    # Write the rows
    writer.writerows(rows)

print(f"Data has been written to {output_file}")

Data has been written to final-date-military-conflict-commander.csv


In [89]:
final_data[0][1]

'leadfigure'

### Date - Commander Mapping

In [6]:
import csv

file_path = 'revised_commander_final.csv'
final_data = []

with open(file_path, mode='r', encoding='utf-8', newline='') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        final_data.append(row)

In [7]:
page_title = []
birth_date = []
death_date = []

start_year = []
start_month = []
start_day = []

end_year = []
end_month = []
end_day = []

for index, value in enumerate(final_data):
    page_title.append(value[4])
    birth_date.append(value[1])
    death_date.append(value[2])
    
    if index == 0 :
        start_year.append('start_year')
        start_month.append('start_month')
        start_day.append('start_day')
        end_year.append('end_year')
        end_month.append('end_month')
        end_day.append('end_day')
    else :
        if len(value[1]) != 0:
            start = value[1].split("-")
            start_year.append(start[0] if start[0] != '00' else '')
            start_month.append(start[1] if start[1] != '00' else '')
            start_day.append(start[2] if start[2] != '00' else '')
        else:
            start_year.append('')
            start_month.append('')
            start_day.append('')

        if len(value[2]) != 0:
            end = value[2].split("-")
            end_year.append(end[0] if end[0] != '00' else '')
            end_month.append(end[1] if end[1] != '00' else '')
            end_day.append(end[2] if end[2] != '00' else '')
        else :
            end_year.append('')
            end_month.append('')
            end_day.append('')

In [8]:
# Combine title_place and place into a list of rows
rows = zip(page_title, birth_date, death_date, start_year, start_month, start_day, end_year, end_month, end_day)

# Specify the column names
fieldnames = ['page_title', 'birth-date', 'death-date', 'start-year', 'start-month', 'start-day', 'end-year', 'end-month', 'end-day']

# Specify the output file path
output_file = 'revised_commander_final_date.csv'

# Write the data to a new CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(fieldnames)
    # Write the rows
    writer.writerows(rows)

print(f"Data has been written to {output_file}")

Data has been written to final-date-military-conflict-commander-revised-with-date.csv
